In [42]:
import numpy as np
import pandas as pd
from decimal import Decimal

# LEFT OFF

You need to figure out what you're doing with `pre_draw_stamp_card_operations`. You were writing `criterion_target_overboost`, but you're wondering if there's still more that needs to be written before you can do that. 

Also, you need to design stamp cards based on the Cloud/Glenn banner. 

### Future Validations

- PullSession
    - `target_weapon_type` is always `featured` or `wishlisted`
    - Number of featured weapons is always 1 or 2
- Stamp cards
    - Ensure `position` is always between 1 and 12.

### Future Refactors

Should a user even have to indicate whether they're targeting a featured or wishlisted weapon? Couldn't we just return the results for both instances?

In [3]:
test_list_of_position_dicts = [
    {
        'position': 4,
         'rule': 'guaranteed_feature_five_star_draw'
    },
    {
        'position': 7,
         'rule': 'guaranteed_feature_five_star_draw'
    },
    {
        'position': 10,
         'rule': 'guaranteed_feature_five_star_draw'
    },
    {
        'position': 12,
         'rule': 'guaranteed_not_desired_draw'
    },
    
]

In [6]:
pd.DataFrame(test_list_of_position_dicts)

,position,rule
0,4,guaranteed_feature_five_star_draw
1,7,guaranteed_feature_five_star_draw
2,10,guaranteed_feature_five_star_draw
3,12,guaranteed_not_desired_draw


In [ ]:
class CrystalPullSession:
    """
    Class representing a pull session. 
    """
    def __init__(self, stamp_cards_list, num_featured_weapons, non_featured_five_star_percent_rate, starting_weapon_parts=0, target_weapon_type):
        
        self.target_weapon_type = target_weapon_type  # In a future refactor, this will be removed, as it will collect data on both types.
        
        self.target_weapon_rates_dict = self.generate_target_probabilities(num_featured_weapons, self.target_weapon_type, non_featured_five_star_percent_rate)

        self.current_stamp_card_index = 0
        self.stamp_cards_list = stamp_cards_list
        self.current_stamp_card = StampCard(self.stamp_cards_list[self.current_stamp_card_index])
        
        self.completed_stamp_cards = []
        self.rules_for_next_ten_draw = []

        self.data = {
            'targeted_weapon_parts': starting_weapon_parts,
            'targeted_five_stars_drawn': 0,
            'targeted_four_stars_drawn': 0,
            'targeted_three_stars_drawn': 0,
            'nontargeted_five_stars_drawn': 0,
            'nontargeted_four_stars_drawn': 0,
            'nontargeted_three_stars_drawn': 0,
        }

    def criterion_target_overboost(self, overboost_target):
        """
        Simulate pulling until reaching a desired overboost level for the targeted weapon. 
        """

        required_weapon_parts = (overboost_target + 1) * 200

        while self.data['targeted_weapon_parts'] < required_weapon_parts:
            self.pre_draw_stamp_card_operations()
            self.perform_ten_draw()

    def determine_stamp_value_for_ten_draw(self, seed=None):
        """
        Generates a number of stamps for the beginning of a 10-draw
        """

        stamp_randint = np.random.default_rng(seed).integers(low=1, high=10000, endpoint=True)
    
        if 1 <= stamp_randint <= 4500:
            stamp_value = self.current_stamp_card.current_stamp_value + 1
        elif 4501 <= stamp_randint <= 8000:
            stamp_value = self.current_stamp_card.current_stamp_value + 2
        elif 8001 <= stamp_randint <= 9592:
            stamp_value = self.current_stamp_card.current_stamp_value + 3
        elif 9593 <= stamp_randint <= 9794:
            stamp_value = self.current_stamp_card.current_stamp_value + 4
        elif 9795 <= stamp_randint <= 9944:
            stamp_value = self.current_stamp_card.current_stamp_value + 5
        elif 9945 <= stamp_randint <= 9999:
            stamp_value = self.current_stamp_card.current_stamp_value + 6
        else: 
            stamp_value = self.current_stamp_card.current_stamp_value + 12

        return stamp_value

    def move_to_next_stamp_card(self):
        """
        Transition the pull session to the next stamp card. 
        """

        self.completed_stamp_cards.append(self.current_stamp_card)
        
        self.current_stamp_card_index += 1

        # Continuously re-use the final (EX) card once all other cards are completed
        if (self.current_stamp_card_index + 1) > len(self.stamp_cards_list):
            self.current_stamp_card = StampCard(self.stamp_cards_list[-1])
        else: 
            self.current_stamp_card = StampCard(self.stamp_cards_list[self.current_stamp_card_index])
    
    def pre_draw_stamp_card_operations(self):
        """
        Determine a stamp value, determine whether a stamp card has been completed based on the result, and
        determine if any stamp card rules need to go into the subsequent ten draw. 
        """

        ten_draw_stamp_value = self.determine_stamp_value_for_ten_draw()

        new_stamp_value = self.current_stamp_card.current_stamp_value + ten_draw_stamp_value
        
        self.log_rules_for_next_draw(new_stamp_value)
        
        if new_stamp_value >= 12:
            self.move_to_next_stamp_card()
            new_stamp_value_for_new_card = new_stamp_value - 12
            self.log_rules_for_next_draw(new_stamp_value_for_new_card)
            self.current_stamp_card.current_stamp_value = new_stamp_value_for_new_card
        else:
            self.current_stamp_card.current_stamp_value = new_stamp_value
    
    def log_rules_for_next_draw(self, new_stamp_value):
        """
        Determines any special rules for a ten draw, based on the current and new stamp card values. 
        """

        for _, row in self.current_stamp_card.position_and_rule_df.iterrows():
            if self.current_stamp_card.current_stamp_value < row['position'] <= new_stamp_value:
                self.rules_for_next_ten_draw.append(row['rule'])
    
    def perform_ten_draw(self):
        """
        Instantiates a TenDraw class object, uses its operations to perform a ten_draw, and stores the results.
        """
        ten_draw = TenDraw(self.rules_for_next_ten_draw, self.target_weapon_rates_dict)

        ten_draw.perform_ten_draw()

        self.data['targeted_weapon_parts'] += ten_draw.pull_results['targeted_weapon_parts']

        for pull_result_string in ten_draw.pull_results['pull_result_strings']:
            for pull_session_outcome in self.data:
                if pull_session_outcome.startswith(pull_result_string):
                    self.data[pull_session_outcome] += 1

    def generate_target_probabilities(num_featured_weapons, target_weapon_type, non_featured_five_star_percent_rate):
        
        """
        Generate a dictionary containing all of the weapon draw rates based on the number of featured weapons and the 
        non-featured five star weapon rate.
        """

        num_weapons_in_banner = round(1.5 / non_featured_five_star_percent_rate) + 5 + num_featured_weapons
    
        if num_featured_weapons != 1:
            raise ValueError("Currently, only banners with one featured weapon are supported.\n You configured for," num_featured_weapons, "weapons.")
        
        if target_weapon_type == 'wishlisted':
            
            target_five_star_rate = 0.01
            target_four_star_rate = Decimal(OVERALL_RARITY_RATES_DICT['four_star'] - ONE_FEATURED_TARGET_FEATURED_RATES_DICT['four_star']) / Decimal(num_weapons_in_banner - num_featured_weapons)
            target_three_star_rate = Decimal(OVERALL_RARITY_RATES_DICT['three_star'] - ONE_FEATURED_TARGET_FEATURED_RATES_DICT['three_star']) / Decimal(num_weapons_in_banner - num_featured_weapons)
            target_guaranteed_four_star_rate = four_star_rate * (OVERALL_RARITY_RATES_DICT['four_star'] + OVERALL_RARITY_RATES_DICT['three_star']) / OVERALL_RARITY_RATES_DICT['four_star']
            
            target_weapon_rates_dict = {
                'five_star': target_five_star_rate,
                'four_star': target_four_star_rate,
                'guaranteed_four_star': target_guaranteed_four_star_rate
                'three_star': target_three_star_rate,
            }

            return target_weapon_rates_dict

        elif target_weapon_type == 'featured':

            return ONE_FEATURED_TARGET_FEATURED_RATES_DICT

In [46]:
class TenDraw:
    """
    Class representing a set of 10 draws within a crystal pull session.
    """

    def __init__(self, rules_for_next_ten_draw, target_weapon_rates_dict):
        self.special_rules = rules_for_next_ten_draw
        self.target_weapon_rates_dict = target_weapon_rates_dict
        self.pull_results = {
            'targeted_weapon_parts': 0,
            'pull_result_strings': [],
        }

    def draw_for_special_rule(self):
        """
        Perform correspending operation for each rule within the special rules list.
        """

        for rule in self.special_rules:
            if rule == 'guaranteed_feature_five_star_draw':
                self.guaranteed_featured_five_star_draw()
            elif rule == 'guaranteed_five_star_draw':
                self.guaranteed_five_star_draw()
            elif rule == 'guaranteed_four_star_draw':
                self.guaranteed_four_star_draw()
            elif rule == 'guaranteed_not_desired_draw':
                self.guaranteed_featured_five_star_draw(desired=False)

    def guaranteed_featured_five_star_draw(self, desired=True):
        """
        Pass a float with value restricted to targeted 5* range through `determine_pull_result()`, but only if the targeted weapon
        is a featured weapon. 
        """

        if desired and self.target_weapon_type == 'featured':
            
            random_float = np.random.default_rng().uniform(0, self.target_weapon_rates_dict['five_star'])
            
            return self.determine_pull_result(random_float)
        else:
            # For now, this will make sure a 5* weapon gets logged. 
            # Will change this once I allow the code to log featured and wishlisted simultaneously.

            random_float = np.random.default_rng().uniform(self.target_weapon_rates_dict['five_star'], OVERALL_RARITY_RATES_DICT['five_star'])
            
            return self.determine_pull_result(random_float)

    def guaranteed_five_star_draw(self, seed=None):
        """
        Pass a float with value restricted to 5* outcomes through `determine_pull_result()`. 
        """

        random_float = np.random.default_rng(seed).uniform(0, OVERALL_RARITY_RATES_DICT['five_star'])
        
        return self.determine_pull_result(random_float)

    def guaranteed_four_star_draw(self, seed=None):
        """
        Pass a float into `determine_pull_result()` and process with all 3* probability rolled into 4* probability.  
        """

        random_float = np.random.default_rng(seed).uniform(0, 1)
        
        return self.determine_pull_result(random_float, guaranteed_four_star=True)
    
    def determine_pull_result(self, random_float, guaranteed_four_star=False):
        """
        Processes the random_float created for a pull and returns the outcome as a string.
        Results are based on the range into which random_float falls. 
        """

        if 0 <= random_float < self.target_weapon_rates_dict['five_star']:
            return 'targeted_five_star'
        elif self.target_weapon_rates_dict['five_star'] <= random_float < OVERALL_RARITY_RATES_DICT['five_star']:
            return 'nontargeted_five_star'
        elif guaranteed_four_star and OVERALL_RARITY_RATES_DICT['five_star'] <= random_float < self.target_weapon_rates_dict['guaranteed_four_star']:
            return 'targeted_four_star'
        elif guaranteed_four_star and self.target_weapon_rates_dict['guaranteed_four_star'] <= random_float < 1:
            return 'nontargeted_four_star'
        elif OVERALL_RARITY_RATES_DICT['five_star'] <= random_float < self.target_weapon_rates_dict['four_star']:
            return 'targeted_four_star'
        elif self.target_weapon_rates_dict['four_star'] <= random_float < OVERALL_RARITY_RATES_DICT['four_star']:
            return 'nontargeted_four_star'
        elif OVERALL_RARITY_RATES_DICT['four_star'] <= random_float < self.target_weapon_rates_dict['three_star']:
            return 'targeted_three_star'
        else:
            return 'nontargeted_three_star'

    def convert_pull_result_to_weapon_parts(pull_result_string):
        """
        Converts a pull result into a number of weapons parts for the targeted weapon.
        """

        if pull_result_string == 'targeted_five_star':
            return 200
        elif pull_result_string == 'targeted_four_star':
            return 10
        elif pull_result_string == 'targeted_three_star':
            return 1
        else:
            return 0

    def standard_single_draw(self, number_of_draws, seed=None):
        """
        Passes a `number_of_draws` random floats through `determine_pull_result()` and returns a list of the result strings. 
        """

        random_floats = np.random.default_rng(seed).uniform(low=0, high=1, size=number_of_draws)

        return [self.determine_pull_result(random_float) for random_float in random_floats]
    
    def perform_ten_draw(self):
        """
        Perform ten draws -- draws with special rules first, and then standard draws until ten have been completed.
        Store the pull results, including result strings and total weapon parts, in the `pull_results` attribute. 
        """

        for special_rule in self.special_rules:
            self.pull_results['pull_result_strings'].append(self.draw_for_special_rule(special_rule))

        number_of_remaining_draws = 10 - len(self.special_rules)
        
        self.pull_results['pull_result_strings'].extend(self.standard_single_draw(number_of_draws=number_of_remaining_draws))

        for pull_result in self.pull_results['pull_result_strings']:
            self.pull_results['targeted_weapon_parts'] += self.convert_pull_result_to_weapon_parts(pull_result)

In [47]:
class StampCard:
    """
    Class representing a single stamp card during a pull session.
    """
    def __init__(self, stamp_card_position_dicts):
        
        self.rule_enum = [
             'guaranteed_feature_five_star_draw',
             'guaranteed_five_star_draw',
             'guaranteed_four_star_draw',
             'guaranteed_not_desired_draw',
        ]

        self.position_and_rule_df = pd.DataFrame(stamp_card_position_dicts)
        
        self.validate_stamp_card_rules()

        self.current_stamp_value = 0

    def validate_stamp_card_rules(self):
        """
        Make sure only supported stamp card rules were provided.
        """
        
        unsupported_stamp_card_rules = []
        
        for rule in self.position_and_rule_df['rule'].drop_duplicates().to_list():
            if rule not in rule_enum:
                unsupported_stamp_card_rules.append(rule)

        if len(unsupported_stamp_card_rules) != 0:
            raise ValueError("One or more unsupported rules in stamp cards", unsupported_stamp_card_rules)

In [59]:
0.05 * (0.225 + 0.70) / 0.225

0.20555555555555555

In [62]:
non_featured_five_star_percent_rate = 0.01339
num_featured_weapons = 2

num_weapons_in_banner = round(1.5 / non_featured_five_star_percent_rate) + 5 + num_featured_weapons

In [63]:
num_weapons_in_banner

119

In [66]:
(0.225 - 0.10) / (num_weapons_in_banner - num_featured_weapons)

0.0010683760683760685

In [64]:
12+10+10+12+10+11+10+11+11+11+11

119